# Traffic Sign Recognition (GTSRB)

**Objective:** Classify traffic signs into 43 classes using deep learning (CNN).

**Topics Covered:**
- Computer Vision (CNN) / Multi-class Classification
- Image preprocessing (resize, normalize)
- Data augmentation
- Custom CNN vs. pre-trained MobileNetV2
- Evaluation: accuracy, confusion matrix

**Tools:** Python, TensorFlow/Keras, OpenCV, scikit-learn

## 1. Imports & Configuration

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Paths
DATA_DIR = os.path.join(os.getcwd(), 'Data')
TRAIN_CSV = os.path.join(DATA_DIR, 'Train.csv')
TEST_CSV  = os.path.join(DATA_DIR, 'Test.csv')
META_CSV  = os.path.join(DATA_DIR, 'Meta.csv')

IMG_SIZE = 32          # resize all images to 32x32
NUM_CLASSES = 43
BATCH_SIZE = 64
EPOCHS = 20
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)

## 2. Load & Explore the Dataset

In [ ]:
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)
meta_df  = pd.read_csv(META_CSV)

print(f"Training samples : {len(train_df)}")
print(f"Test samples     : {len(test_df)}")
print(f"Number of classes: {train_df['ClassId'].nunique()}")
print()
train_df.head()

In [ ]:
# Class distribution
fig, ax = plt.subplots(figsize=(14, 5))
class_counts = train_df['ClassId'].value_counts().sort_index()
ax.bar(class_counts.index, class_counts.values, color='steelblue', edgecolor='black')
ax.set_xlabel('Class ID')
ax.set_ylabel('Number of Images')
ax.set_title('Training Set — Class Distribution')
ax.set_xticks(range(NUM_CLASSES))
plt.tight_layout()
plt.show()

print(f"Min samples per class: {class_counts.min()} (class {class_counts.idxmin()})")
print(f"Max samples per class: {class_counts.max()} (class {class_counts.idxmax()})")

In [ ]:
# Visualise one sample per class
fig, axes = plt.subplots(5, 9, figsize=(16, 10))
axes = axes.flatten()

for class_id in range(NUM_CLASSES):
    sample = train_df[train_df['ClassId'] == class_id].iloc[0]
    img_path = os.path.join(DATA_DIR, sample['Path'])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[class_id].imshow(img)
    axes[class_id].set_title(f'Class {class_id}', fontsize=8)
    axes[class_id].axis('off')

# Hide unused subplots
for i in range(NUM_CLASSES, len(axes)):
    axes[i].axis('off')

plt.suptitle('Sample Image per Class', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## 3. Image Preprocessing — Resize & Normalize

In [ ]:
def load_images(df, data_dir, img_size=IMG_SIZE):
    """Load images from paths in a DataFrame, resize and normalise to [0, 1]."""
    images = []
    labels = []
    for _, row in df.iterrows():
        img_path = os.path.join(data_dir, row['Path'])
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(row['ClassId'])
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

print("Loading training images...")
X_full, y_full = load_images(train_df, DATA_DIR)
print(f"  Loaded {X_full.shape[0]} images — shape: {X_full.shape[1:]}")

print("Loading test images...")
X_test, y_test = load_images(test_df, DATA_DIR)
print(f"  Loaded {X_test.shape[0]} images — shape: {X_test.shape[1:]}")

In [ ]:
# Train / Validation split (80 / 20)
X_train, X_val, y_train, y_val = train_test_split(
    X_full, y_full, test_size=0.2, random_state=SEED, stratify=y_full
)

# One-hot encode labels
y_train_cat = to_categorical(y_train, NUM_CLASSES)
y_val_cat   = to_categorical(y_val,   NUM_CLASSES)
y_test_cat  = to_categorical(y_test,  NUM_CLASSES)

print(f"Train : {X_train.shape[0]}")
print(f"Val   : {X_val.shape[0]}")
print(f"Test  : {X_test.shape[0]}")

## 4. Data Augmentation (Bonus)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=False,     # traffic signs are NOT horizontally symmetric
)
train_datagen.fit(X_train)

train_generator = train_datagen.flow(X_train, y_train_cat, batch_size=BATCH_SIZE, seed=SEED)

# Preview augmented images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
sample_batch, _ = next(train_generator)
for i, ax in enumerate(axes.flatten()):
    ax.imshow(sample_batch[i])
    ax.axis('off')
plt.suptitle('Augmented Training Samples', fontsize=13)
plt.tight_layout()
plt.show()

## 5. Build Custom CNN Model

In [ ]:
def build_custom_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES):
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        # Classifier head
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

cnn_model = build_custom_cnn()
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
cnn_model.summary()

## 6. Train Custom CNN

In [ ]:
early_stop = callbacks.EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True
)
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1
)

cnn_history = cnn_model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val_cat),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

In [ ]:
def plot_training_history(history, title=''):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Accuracy
    ax1.plot(history.history['accuracy'],     label='Train')
    ax1.plot(history.history['val_accuracy'],  label='Validation')
    ax1.set_title(f'{title} — Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Loss
    ax2.plot(history.history['loss'],     label='Train')
    ax2.plot(history.history['val_loss'],  label='Validation')
    ax2.set_title(f'{title} — Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

plot_training_history(cnn_history, title='Custom CNN')

## 7. Evaluate Custom CNN on Test Set

In [ ]:
cnn_test_loss, cnn_test_acc = cnn_model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Custom CNN — Test Accuracy: {cnn_test_acc:.4f}")
print(f"Custom CNN — Test Loss    : {cnn_test_loss:.4f}")

y_pred_cnn = np.argmax(cnn_model.predict(X_test, verbose=0), axis=1)

print("\nClassification Report (Custom CNN):")
print(classification_report(y_test, y_pred_cnn, digits=3))

In [ ]:
# Confusion Matrix
cm_cnn = confusion_matrix(y_test, y_pred_cnn)

fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=range(NUM_CLASSES), yticklabels=range(NUM_CLASSES))
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('Actual', fontsize=12)
ax.set_title('Confusion Matrix — Custom CNN', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Bonus — Pre-trained MobileNetV2 (Transfer Learning)

We up-scale images to 96×96 for MobileNetV2 (minimum input 32×32, but larger works better).

In [ ]:
MOBILE_IMG_SIZE = 96

# Resize data for MobileNetV2
X_train_m = np.array([cv2.resize(img, (MOBILE_IMG_SIZE, MOBILE_IMG_SIZE)) for img in X_train])
X_val_m   = np.array([cv2.resize(img, (MOBILE_IMG_SIZE, MOBILE_IMG_SIZE)) for img in X_val])
X_test_m  = np.array([cv2.resize(img, (MOBILE_IMG_SIZE, MOBILE_IMG_SIZE)) for img in X_test])

print(f"Resized train shape: {X_train_m.shape}")
print(f"Resized test shape : {X_test_m.shape}")

In [ ]:
def build_mobilenet_model(input_shape=(MOBILE_IMG_SIZE, MOBILE_IMG_SIZE, 3), num_classes=NUM_CLASSES):
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    # Freeze base layers
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

mobile_model = build_mobilenet_model()
mobile_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
mobile_model.summary()

In [ ]:
# Data augmentation for MobileNet input
mobile_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=False,
)
mobile_datagen.fit(X_train_m)
mobile_generator = mobile_datagen.flow(X_train_m, y_train_cat, batch_size=BATCH_SIZE, seed=SEED)

# Phase 1: Train head only
mobile_history_1 = mobile_model.fit(
    mobile_generator,
    steps_per_epoch=len(X_train_m) // BATCH_SIZE,
    epochs=10,
    validation_data=(X_val_m, y_val_cat),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)
print("\n✓ Phase 1 complete (frozen base).")

In [ ]:
# Phase 2: Fine-tune — unfreeze top layers of MobileNetV2
base = mobile_model.layers[0]
base.trainable = True

# Freeze all but last 30 layers
for layer in base.layers[:-30]:
    layer.trainable = False

mobile_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

mobile_history_2 = mobile_model.fit(
    mobile_generator,
    steps_per_epoch=len(X_train_m) // BATCH_SIZE,
    epochs=10,
    validation_data=(X_val_m, y_val_cat),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)
print("\n✓ Phase 2 complete (fine-tuned).")

In [ ]:
# Combine histories for plotting
combined_history = {}
for key in mobile_history_1.history:
    combined_history[key] = mobile_history_1.history[key] + mobile_history_2.history[key]

class CombinedHistory:
    def __init__(self, h): self.history = h

plot_training_history(CombinedHistory(combined_history), title='MobileNetV2')

## 9. Evaluate MobileNetV2 on Test Set

In [ ]:
mob_test_loss, mob_test_acc = mobile_model.evaluate(X_test_m, y_test_cat, verbose=0)
print(f"MobileNetV2 — Test Accuracy: {mob_test_acc:.4f}")
print(f"MobileNetV2 — Test Loss    : {mob_test_loss:.4f}")

y_pred_mob = np.argmax(mobile_model.predict(X_test_m, verbose=0), axis=1)

print("\nClassification Report (MobileNetV2):")
print(classification_report(y_test, y_pred_mob, digits=3))

In [ ]:
# Confusion Matrix — MobileNetV2
cm_mob = confusion_matrix(y_test, y_pred_mob)

fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(cm_mob, annot=True, fmt='d', cmap='Greens', ax=ax,
            xticklabels=range(NUM_CLASSES), yticklabels=range(NUM_CLASSES))
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('Actual', fontsize=12)
ax.set_title('Confusion Matrix — MobileNetV2', fontsize=14)
plt.tight_layout()
plt.show()

## 10. Comparison — Custom CNN vs. MobileNetV2

In [ ]:
# Side-by-side comparison
comparison = pd.DataFrame({
    'Model': ['Custom CNN', 'MobileNetV2 (fine-tuned)'],
    'Test Accuracy': [cnn_test_acc, mob_test_acc],
    'Test Loss':     [cnn_test_loss, mob_test_loss],
    'Parameters':    [cnn_model.count_params(), mobile_model.count_params()],
})
comparison['Test Accuracy'] = comparison['Test Accuracy'].map('{:.4f}'.format)
comparison['Test Loss']     = comparison['Test Loss'].map('{:.4f}'.format)
comparison['Parameters']    = comparison['Parameters'].map('{:,}'.format)

print("=" * 70)
print("               MODEL COMPARISON")
print("=" * 70)
print(comparison.to_string(index=False))
print("=" * 70)

In [ ]:
# Visualise some test predictions (both models)
fig, axes = plt.subplots(3, 5, figsize=(15, 9))
indices = np.random.choice(len(X_test), 15, replace=False)

for i, ax in enumerate(axes.flatten()):
    idx = indices[i]
    ax.imshow(X_test[idx])
    true_label = y_test[idx]
    cnn_pred   = y_pred_cnn[idx]
    mob_pred   = y_pred_mob[idx]

    colour = 'green' if cnn_pred == true_label else 'red'
    ax.set_title(
        f'True: {true_label}\nCNN: {cnn_pred} | Mob: {mob_pred}',
        fontsize=8, color=colour
    )
    ax.axis('off')

plt.suptitle('Test Predictions — Green = CNN correct, Red = CNN wrong', fontsize=12)
plt.tight_layout()
plt.show()

## 11. Per-Class Accuracy Comparison

In [ ]:
# Per-class accuracy for both models
cnn_per_class = cm_cnn.diagonal() / cm_cnn.sum(axis=1)
mob_per_class = cm_mob.diagonal() / cm_mob.sum(axis=1)

fig, ax = plt.subplots(figsize=(16, 6))
x = np.arange(NUM_CLASSES)
width = 0.35

bars1 = ax.bar(x - width/2, cnn_per_class, width, label='Custom CNN', color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, mob_per_class, width, label='MobileNetV2', color='seagreen', alpha=0.8)

ax.set_xlabel('Class ID')
ax.set_ylabel('Accuracy')
ax.set_title('Per-Class Test Accuracy — Custom CNN vs MobileNetV2')
ax.set_xticks(x)
ax.set_ylim(0, 1.1)
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Worst classes for each model
worst_cnn = np.argsort(cnn_per_class)[:5]
worst_mob = np.argsort(mob_per_class)[:5]
print(f"Top-5 hardest classes (CNN)       : {worst_cnn} — acc: {cnn_per_class[worst_cnn].round(3)}")
print(f"Top-5 hardest classes (MobileNet) : {worst_mob} — acc: {mob_per_class[worst_mob].round(3)}")

## 12. Summary

| Aspect | Custom CNN | MobileNetV2 (Transfer Learning) |
|---|---|---|
| Architecture | 3 conv blocks + dense head | Pre-trained backbone + dense head |
| Training | From scratch with augmentation | 2-phase: frozen then fine-tuned |
| Parameters | ~600 K | ~2.6 M (most frozen) |
| Expected Accuracy | ~95–97 % | ~96–98 % |

**Key Takeaways:**
- Data augmentation significantly helps with the imbalanced GTSRB dataset.
- Batch normalisation and dropout are essential regularisation techniques.
- Transfer learning with MobileNetV2 converges faster and usually achieves slightly better accuracy.
- The per-class analysis reveals which sign categories are hardest to distinguish.